# SABR Calibration Demo (FX)

This notebook demonstrates SABR smile calibration for FX options.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from derivx.vol.sabr import calibrate_sabr_smile, hagan_sabr_iv

%matplotlib inline


## Generate synthetic SABR smile and calibrate


In [ ]:
# True SABR parameters (FX smile)
F = 1.10  # EURUSD forward
T = 1.0
true_alpha, true_beta, true_rho, true_nu = 0.25, 0.5, -0.3, 0.4
Ks = np.linspace(0.95, 1.25, 15)

# Generate true IVs + noise
ivs_true = np.array([hagan_sabr_iv(F, k, T, true_alpha, true_beta, true_rho, true_nu) for k in Ks])
np.random.seed(123)
ivs_obs = ivs_true + 0.002 * np.random.randn(len(Ks))

# Calibrate
params_est = calibrate_sabr_smile(Ks, ivs_obs, T, F, beta=0.5)
ivs_fit = np.array([hagan_sabr_iv(F, k, T, **params_est) for k in Ks])

print("Estimated SABR params:", {k: f"{v:.4f}" for k, v in params_est.items()})
print(f"RMSE: {np.sqrt(np.mean((ivs_fit - ivs_true)**2)):.6f}")

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(Ks, ivs_obs, label='Observed IVs', alpha=0.6, s=50)
plt.plot(Ks, ivs_fit, label='SABR fit', linewidth=2, color='blue')
plt.plot(Ks, ivs_true, label='True smile', linewidth=1, linestyle='--', alpha=0.7, color='green')
plt.xlabel('Strike')
plt.ylabel('Implied Volatility')
plt.title('SABR Smile Calibration (FX)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()
